In [2]:
import torch
import os
#from ch7.weather.model.model import TcnClassifier
#from ch7.weather.utils import sliding_window
import pandas as pd
import numpy as np
import random
import copy


def load_data():    
    
    raw_df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQwC6jRtVUk-2dkk2W3BDJZTOdsS427LN8Ixo-rQF4Afs6ice0rof7qh_EbnAy5lYEGqX-TCSvjpPyr/pub?gid=1713335339&single=true&output=csv',
                        index_col=['codigo','Tiempo']).drop(['Fecha','Exposicion'], axis=1)
    raw_df.sort_index(inplace=True)
    #raw_df['adherence'] = np.where((raw_df['SA'] < 0.5) & (raw_df['Tirosina'] < 600) & (raw_df['NTBC_DBS'] > 15), 'Yes', 'No')
    
    raw_df['alfafeto_bad'] = np.where((raw_df['Alfa-fetoprot'] > 10), 'Yes', 'No')
    
    #raw_df['adherence']
    df_sin_index = raw_df.reset_index()
    assert 'Alfa-fetoprot' in df_sin_index.columns.to_list()
    return df_sin_index

original_data = load_data()


original_data

,codigo,Tiempo,Dosis NTBC (mg/kg),NTBC_DBS,SA,Metionina,Tirosina,Fenilalanina,Relación Tirosina/Fenilalanina,INR,...,Butirilcarnitina,Glutarilcarnitina,Hexanoilcarnitina,Octanoilcarnitina,Dodecanoilcarnitina,Tetradecenoilcarnitna,Palmitoilcarnitina,Estearoilcarnitina,3-OH-Linoleoilcarnitina,alfafeto_bad
0,T01,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
1,T01,2,1.940,35.35,0.0,31.30,249.70,24.00,10.400000,1.22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
2,T01,3,1.190,48.80,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
3,T01,4,1.120,38.40,NaN,20.18,202.21,8.62,23.458237,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
4,T01,5,NaN,12.58,NaN,17.09,280.75,19.90,14.108040,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,T19,5,0.738,NaN,NaN,NaN,NaN,NaN,NaN,1.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
167,T19,6,0.630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
168,T19,7,0.640,20.34,NaN,15.25,268.83,58.13,4.624634,NaN,...,0.17,0.07,0.01,0.08,0.05,0.08,0.89,0.44,0.08,No
169,T19,8,0.630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No


In [2]:
raw_df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQwC6jRtVUk-2dkk2W3BDJZTOdsS427LN8Ixo-rQF4Afs6ice0rof7qh_EbnAy5lYEGqX-TCSvjpPyr/pub?gid=1713335339&single=true&output=csv',
                     index_col=['codigo','Tiempo']).drop(['Fecha','Exposicion'], axis=1)

raw_df.sort_index(inplace=True)
timepoints_per_patient = pd.Series(raw_df.index.get_level_values(0).values).value_counts()
smallest_timeseries        = timepoints_per_patient.min()
df                     = raw_df.loc[(slice(None), slice(1, smallest_timeseries)), :]
print(smallest_timeseries, len(df.columns))



imputed_train_df =  abs(pd.read_csv('imputed_train_df.csv', index_col=['codigo']).drop('Unnamed: 0', axis=1))#.sort_index(inplace=False))
imputed_val_df   =   abs(pd.read_csv('imputed_val_df.csv', index_col=['codigo']).drop('Unnamed: 0', axis=1))#.sort_index(inplace=False))
imputed_df       = pd.concat([imputed_val_df, imputed_train_df])#.sort_index(inplace=False)
IDs              =  imputed_df.index.get_level_values(0).unique()

timepoints_per_patient = pd.Series(raw_df.index.get_level_values(0).values).value_counts()
timeseries_length      =  timepoints_per_patient.min()
import itertools

#muliindex = itertools.product(IDs, range(1, timeseries_length+1))
imputed_df.index = pd.MultiIndex.from_product([IDs, range(1, timeseries_length+1)], names=["code", "time"])
imputed_df#.sort_index(inplace=True)
df = imputed_df
df.columns = raw_df.columns
df['alfafeto_bad'] = np.where((df['Alfa-fetoprot'] > 10), 'Yes', 'No')

print(df['alfafeto_bad'].value_counts())
df_sin_index = raw_df.reset_index()



#df_sin_index.drop( 'Alfa-fetoprot', axis=1, inplace=True)

#features_continuos = 
assert 'Alfa-fetoprot' in df_sin_index.columns.to_list()

9 51
No     113
Yes     49
Name: alfafeto_bad, dtype: int64


In [4]:
def prepare_and_train_model(params, epochs, save_model = False, model_name = 'model'):
    dir_path = os.path.dirname(os.path.realpath('__file__')) #__file__ solo funciona para la terminal. Para notebook usar ''
    
    # Explicitly define the end date
    #end_date = '2016-01-01'
    # Historical data
    #df = get_df_until_2016_01_01()
    #df = df[df.index < end_date]
    df = load_data()
    # 3 timepoints as sliding window
    w = 3
    # Number of epochs for training
    #epochs = 50 

    # Hyper-parameters:
    tcl_num = params['tcl_num']
    tcl_channel_size = params['tcl_channel_size']
    # temporal casual layer channels
    channel_sizes = [tcl_channel_size] * tcl_num
    # convolution kernel size
    kernel_size = params['kernel_size']
    dropout = params['dropout']
    slices = params['slices']
    use_bias = params['use_bias']
    lr = params['lr']

    # Australia Location for training
    #patients_list = list(raw_df.index.get_level_values(0).unique())
    codigos =['T01','T02','T03','T04','T05','T06','T07','T08', 'T09', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'T17', 'T18', 'T19']
    #locations = ['Albury', 'Newcastle', 'Richmond', 'Sydney', 'Canberra']
    # features
    features_cont = ['Dosis NTBC (mg/kg)', 'NTBC_DBS', 'SA',
    'Metionina', 'Tirosina', 'Fenilalanina', 'Relación Tirosina/Fenilalanina', 'INR', 'PT (seg)', 'Bili Total', 'Bili Directa', 'GPT', 'GOT', 'GGT',
    'FosfatasaAlcalina','Alfa-fetoprot', 'Glicemia', 'Alanina', 'Aspartato', 'Glutamato', 'Leucina', 'Ornitina', 'Prolina', 'Tirosina_GSS', 'Carnitina libre', 'Propionilcarnitina', 'Isovalerilcarnitina', 'Tiglilcarnitina',
    'Me-Glutarilcarnitina', 'Decanoilcarnitina', 'Tetradecanoilcarnitna', '3-OH-Isovalerilcarnitina', '3-OH-Palmiltoilcarnitna', 'Linoleoilcarnitina', 'Arginina', 'Citrulina', 'Glicina', 'Metionina_GSS', 'Fenilalanina_GSS', 'SA_GSS', 'Valina', 'Acetilcarnitina', 'Butirilcarnitina', 'Glutarilcarnitina', 'Hexanoilcarnitina',
    'Octanoilcarnitina', 'Dodecanoilcarnitina', 'Tetradecenoilcarnitna', 'Palmitoilcarnitina', 'Estearoilcarnitina', '3-OH-Linoleoilcarnitina']
 
    features_cat = ['alfafeto_bad']
    #features_cont = ['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed', 'WindSpeed9am',
    #                 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am',
    #                 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm']
    #features_cat = ['RainToday']

    X, Y = [], []
    for l in codigos:
        df_l = df[df['codigo'] == l]
        D = []
        for f in features_cont:
            D.append(df_l[f].interpolate('linear').fillna(0).values)
        for f in features_cat:
            D.append(df_l[f].map({'Yes': 1, 'No': 0}).fillna(0).values)
            # transpose to time series
        TS = []
        for i in range(df_l.shape[0]):
            row = []
            for c in D:
                row.append(c[i])
            TS.append(row)
        in_seq, out_seq = sliding_window(TS, w, 1)
        rain_seq = [r[0][-1] for r in out_seq]
        X.extend(in_seq)
        Y.extend(rain_seq)

    # Train-Validation Split
    X_train, Y_train = [], []
    X_val, Y_val = [], []
    for i in range(len(X)):
        if random.random() > .8:
            X_val.append(X[i])
            Y_val.append(Y[i])
        else:
            X_train.append(X[i])
            Y_train.append(Y[i])

    x_train = torch.tensor(X_train).float().transpose(1, 2)
    y_train = torch.tensor(Y_train).long()
    x_val = torch.tensor(X_val).float().transpose(1, 2)
    y_val = torch.tensor(Y_val).long()

    model_params = {
        'num_inputs':   len(features_cont) + len(features_cat),
        'num_classes':  2,
        'num_channels': channel_sizes,
        'kernel_size':  kernel_size,
        'dropout':      dropout,
        'slices':       slices,
        'act':          'relu',
        'use_bias':     use_bias
    }
    model = TcnClassifier(**model_params)

    optimizer = torch.optim.Adam(params = model.parameters(), lr = lr)
    cl_loss = torch.nn.CrossEntropyLoss()

    best_params = None
    min_val_loss = 1000_000

    training_loss = []
    validation_loss = []

    for t in range(epochs):

        prediction = model(x_train)
        loss = cl_loss(prediction, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        val_prediction = model(x_val)
        val_loss = cl_loss(val_prediction, y_val)

        training_loss.append(loss.item())
        validation_loss.append(val_loss.item())

        if val_loss.item() < min_val_loss:
            best_params = copy.deepcopy(model.state_dict())
            min_val_loss = val_loss.item()
            if save_model:
                torch.save(best_params, f'{dir_path}/data/{model_name}.pt')
                
                
                
                print(f'best model updated and saved, new min_val_loss: {min_val_loss}')

        if t % 1000 == 0:
            print(f'Epoch {t}| test: {round(loss.item(),4)}, '
                  f'val: {round(val_loss.item(),4)}')
            
    all_data = torch.cat((x_train, x_val))
    all_Ys   = torch.cat((y_train, y_val))


    return min_val_loss, model_params, all_data, all_Ys


In [5]:
from ch7.weather.model.model import TcnClassifier
from ch7.weather.utils import sliding_window



params = {"tcl_num":          4,
        "tcl_channel_size": 25,
        "kernel_size":      4,
        "dropout":          0.1,
        "slices":           1,
        "use_bias":         True,
        "lr":                0.03388437885302203}

min_val_loss, model_params, all_data, all_Ys = prepare_and_train_model(params, epochs = 10000, save_model = True, model_name = 'tcn_tirosinemia_v2')

best model updated and saved, new min_val_loss: 931.8802490234375
Epoch 0| test: 15.0479, val: 931.8802
best model updated and saved, new min_val_loss: 110.65357208251953
best model updated and saved, new min_val_loss: 5.874084949493408
best model updated and saved, new min_val_loss: 3.844174861907959
best model updated and saved, new min_val_loss: 1.3523058891296387
best model updated and saved, new min_val_loss: 1.1579686403274536
best model updated and saved, new min_val_loss: 1.0827295780181885
best model updated and saved, new min_val_loss: 0.7924841046333313
best model updated and saved, new min_val_loss: 0.6747756600379944
best model updated and saved, new min_val_loss: 0.6735140681266785
best model updated and saved, new min_val_loss: 0.6696830987930298
best model updated and saved, new min_val_loss: 0.6485798358917236
best model updated and saved, new min_val_loss: 0.6318516135215759
best model updated and saved, new min_val_loss: 0.6039059162139893
best model updated and save

In [5]:
best_model                 = TcnClassifier(**model_params)
saved_parameters_path = f"{os.getcwd()}/data/tcn_tirosinemia_v2.pt"
best_model.load_state_dict(torch.load(saved_parameters_path));
torch.save(best_model, "data/best_model.pt")

In [6]:
import pickle
learning_data = {'all_data': all_data, 'all_Ys': all_Ys}



with open('data/learning_data.pickle', 'wb') as handle:
    pickle.dump(learning_data, handle, protocol=pickle.HIGHEST_PROTOCOL)